# AMP-SOA portfolio optimization using adaptive meta-policy based on second-order agents with deep reinforcment learning
----

## 3.0 Stock Selection
---
Auto Encoders are employed to select the less volatile stocks by choosing stocks with less reconstruction error. These are the stocks that are included in the portfolio

### 3.1 Import Relevant Libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras

In [2]:
from numpy import array
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense, Activation
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.utils import plot_model
from keras import regularizers, optimizers

from sklearn import preprocessing

### 3.2 Load the Data

In [3]:
# Load the close prices dataset
prices_data = pd.read_csv('./datasets/close_prices.csv')

In [4]:
df = prices_data.copy()

In [5]:
df = df.reset_index(drop=True).set_index(['date'])

In [6]:
df.head()

,CAT,MMM,PG,BA,VZ,WMT,GS,DD,INTC,TRV,...,IBM,KO,JNJ,XOM,AXP,HD,RTX,V,DIS,MSFT
date,,,,,,,,,,,,,,,,,,,,,
2008-03-19,47.481430,40.100849,42.133179,54.094528,14.193682,11.901863,127.170723,32.205429,13.014485,31.220446,...,64.913170,18.092833,40.176636,46.908852,32.275002,17.789188,29.209347,12.605216,26.296064,20.830700
2008-03-20,47.539413,39.348988,43.102489,55.088776,14.593674,12.476101,137.207535,32.267361,13.421768,32.148785,...,65.684784,18.418718,40.486256,47.225517,35.333435,18.597795,29.315559,14.356564,26.851618,21.238287
2008-03-24,48.962597,39.682026,43.475292,56.186153,14.937099,12.569845,136.634583,33.311092,13.656260,32.081516,...,66.089989,18.460970,40.182823,47.753345,36.432312,19.393135,29.880531,13.325836,26.969458,21.231007
2008-03-25,49.355450,39.853596,43.208122,55.898914,14.908816,12.433908,137.207535,33.487988,13.742653,32.128609,...,65.484924,18.530369,39.972290,47.336658,36.509171,19.061741,29.804073,14.111153,27.003124,21.209167
2008-03-26,49.819111,39.692108,43.270245,56.193516,14.601750,12.398750,134.037613,33.443768,13.489645,31.987318,...,64.896507,18.451914,40.065163,47.925556,34.857006,18.664076,29.570442,14.269552,26.733772,20.787031


### 3.3 Define Functions

In [7]:
def defineAutoencoder(num_stock, encoding_dim = 5, verbose=0):
    
    """
    Function for fitting an Autoencoder
    """

    # connect all layers
    input = Input(shape=(num_stock,))

    encoded = Dense(encoding_dim, kernel_regularizer=regularizers.l2(0.00001),name ='Encoder_Input')(input)

    decoded = Dense(num_stock, kernel_regularizer=regularizers.l2(0.00001), name ='Decoder_Input')(encoded)
    decoded = Activation("linear", name='Decoder_Activation_function')(decoded)

    # construct and compile AE model
    autoencoder = Model(inputs=input, outputs=decoded)
    adam = optimizers.Adam(learning_rate=0.0005)
    autoencoder.compile(optimizer=adam, loss='mean_squared_error')
    if verbose!= 0:
        autoencoder.summary()

    return autoencoder

In [8]:
def getReconstructionErrorsDF(df_pct_change, reconstructed_data):
    
    """
    Function for calculating the reconstruction Errors
    """
    array = []
    stocks_ranked = []
    num_columns = reconstructed_data.shape[1]
    for i in range(0, num_columns):
        diff = np.linalg.norm((df_pct_change.iloc[:, i] - reconstructed_data[:, i]))  # 2 norm difference
        array.append(float(diff))

    ranking = np.array(array).argsort()
    r = 1
    for stock_index in ranking:
        stocks_ranked.append([ r
                              ,stock_index
                              ,df_pct_change.iloc[:, stock_index].name
                              ,array[stock_index]
                              ])
        r = r + 1

    columns = ['ranking','stock_index', 'stock_name' ,'recreation_error']
    df = pd.DataFrame(stocks_ranked, columns=columns)
    df = df.set_index('stock_name')
    return df

### 3.4 Get the Percentage Change of the Close Prices

In [9]:
col_names = df.columns.to_list()

In [10]:
print(col_names)

['CAT', 'MMM', 'PG', 'BA', 'VZ', 'WMT', 'GS', 'DD', 'INTC', 'TRV', 'MRK', 'MCD', 'CSCO', 'JPM', 'NKE', 'CVX', 'UNH', 'WBA', 'AAPL', 'PFE', 'IBM', 'KO', 'JNJ', 'XOM', 'AXP', 'HD', 'RTX', 'V', 'DIS', 'MSFT']


In [11]:
df_pct_change = df.pct_change(1).astype(float)
df_pct_change = df_pct_change.replace([np.inf, -np.inf], np.nan)
df_pct_change = df_pct_change.fillna(method='ffill')

# the percentage change function will make the first two rows equal to nan
df_pct_change = df_pct_change.tail(len(df_pct_change) - 2)

C:\Users\nice\AppData\Local\Temp\ipykernel_2332\3045634350.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_pct_change = df_pct_change.fillna(method='ffill')


In [12]:
df_pct_change.shape

(4033, 30)

In [13]:
# remove columns where there is no change over a longer time period
df_pct_change = df_pct_change[df_pct_change.columns[((df_pct_change == 0).mean() <= 0.05)]]

In [14]:
df_pct_change.head()

,CAT,MMM,PG,BA,VZ,WMT,GS,DD,INTC,TRV,...,IBM,KO,JNJ,XOM,AXP,HD,RTX,V,DIS,MSFT
date,,,,,,,,,,,,,,,,,,,,,
2008-03-24,0.029937,0.008464,0.008649,0.019920,0.023533,0.007514,-0.004176,0.032346,0.017471,-0.002092,...,0.006169,0.002294,-0.007495,0.011177,0.031100,0.042765,0.019272,-0.071795,0.004389,-0.000343
2008-03-25,0.008024,0.004324,-0.006145,-0.005112,-0.001893,-0.010815,0.004193,0.005310,0.006326,0.001468,...,-0.009155,0.003759,-0.005239,-0.008726,0.002110,-0.017088,-0.002559,0.058932,0.001248,-0.001029
2008-03-26,0.009394,-0.004052,0.001438,0.005270,-0.020596,-0.002828,-0.023103,-0.001320,-0.018410,-0.004398,...,-0.008986,-0.004234,0.002323,0.012441,-0.045253,-0.020862,-0.007839,0.011225,-0.009975,-0.019903
2008-03-27,-0.004266,-0.010170,-0.003302,-0.027261,-0.004981,-0.010019,-0.041828,-0.013640,-0.035224,-0.001892,...,-0.011889,0.002616,-0.001236,-0.000695,-0.011685,-0.013140,-0.002873,-0.016417,-0.011965,-0.017857
2008-03-28,0.000779,0.001156,0.000432,-0.010105,-0.003059,-0.004774,-0.021946,-0.005965,-0.014225,-0.000843,...,-0.008224,-0.006035,-0.006809,-0.011369,-0.037474,-0.014753,-0.012390,-0.002384,-0.012110,-0.004991


### 3.5 Construct the Autoencoder

In [15]:
# define the input parameters
hidden_layers = 5
batch_size = 300
epochs = 500
stock_selection_number = 20
num_stock = df_pct_change.shape[1]
verbose = 1

In [16]:
print('-' * 20 + 'Step 1 : Returns vs. recreation error (recreation_error)')
print('-' * 25 + 'Transform dataset with MinMax Scaler')

--------------------Step 1 : Returns vs. recreation error (recreation_error)
-------------------------Transform dataset with MinMax Scaler


In [17]:
# Normalize the data
df_scaler = preprocessing.MinMaxScaler()
df_pct_change_normalised = df_scaler.fit_transform(df_pct_change)

In [18]:
# define autoencoder
print('-' * 25 + 'Define autoencoder model')
num_stock = len(df_pct_change.columns)
autoencoder = defineAutoencoder(num_stock=num_stock, encoding_dim=hidden_layers, verbose=verbose)
#plot_model(autoencoder, to_file='img/model_autoencoder_1.png', show_shapes=True,
#           show_layer_names=True)

-------------------------Define autoencoder model

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 30)]              0         
                                                                 
 Encoder_Input (Dense)       (None, 5)                 155       
                                                                 
 Decoder_Input (Dense)       (None, 30)                180       
                                                                 
 Decoder_Activation_functio  (None, 30)                0         
 n (Activation)                                                  
                                                                 
Total params: 335 (1.31 KB)
Trainable params: 335 (1.31 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
# train autoencoder
print('-' * 25 + 'Train autoencoder model')
autoencoder.fit(df_pct_change_normalised, df_pct_change_normalised, shuffle=True, epochs=epochs,batch_size=batch_size,verbose=verbose)

-------------------------Train autoencoder model
Epoch 1/500

14/14 [==============================] - 0s 1ms/step - loss: 0.3354
Epoch 2/500
14/14 [==============================] - 0s 1ms/step - loss: 0.2843
Epoch 3/500
14/14 [==============================] - 0s 1ms/step - loss: 0.2426
Epoch 4/500
14/14 [==============================] - 0s 1ms/step - loss: 0.2091
Epoch 5/500
14/14 [==============================] - 0s 1ms/step - loss: 0.1818
Epoch 6/500
14/14 [==============================] - 0s 1ms/step - loss: 0.1592
Epoch 7/500
14/14 [==============================] - 0s 1ms/step - loss: 0.1401
Epoch 8/500
14/14 [==============================] - 0s 1ms/step - loss: 0.1236
Epoch 9/500
14/14 [==============================] - 0s 1ms/step - loss: 0.1092
Epoch 10/500
14/14 [==============================] - 0s 0s/step - loss: 0.0963
Epoch 11/500
14/14 [==============================] - 0s 0s/step - loss: 0.0848
Epoch 12/500
14/14 [==============================] - 0s 74us/step - l

In [20]:
# predict autoencoder
print('-' * 25 + 'Predict autoencoder model')
reconstruct = autoencoder.predict(df_pct_change_normalised)

-------------------------Predict autoencoder model
127/127 [==============================] - 0s 662us/step


In [21]:
# Inverse transform dataset with MinMax Scaler
print('-' * 25 + 'Inverse transform dataset with MinMax Scaler')
reconstruct_real = df_scaler.inverse_transform(reconstruct)
df_reconstruct_real = pd.DataFrame(data=reconstruct_real, columns=df_pct_change.columns)

-------------------------Inverse transform dataset with MinMax Scaler


In [22]:
print('-' * 25 + 'Calculate L2 norm as reconstruction loss metric')
df_recreation_error = getReconstructionErrorsDF(df_pct_change=df_pct_change,
                                                reconstructed_data=reconstruct_real)

-------------------------Calculate L2 norm as reconstruction loss metric


In [23]:
df_recreation_error

,ranking,stock_index,recreation_error
stock_name,,,
PG,1,2,0.441039
JNJ,2,22,0.468180
VZ,3,4,0.519594
KO,4,21,0.544595
PFE,5,19,0.582681
MCD,6,11,0.583019
MMM,7,1,0.592029
IBM,8,20,0.620907
WMT,9,5,0.640834


In [24]:
filtered_stocks = df_recreation_error.head(stock_selection_number).index

In [25]:
filtered_stocks
my_filtered_stocks = pd.DataFrame(filtered_stocks)
my_filtered_stocks.to_csv('filtered_stocks.csv')

In [26]:
# store the list of selected stocks
%store filtered_stocks

Stored 'filtered_stocks' (Index)
